In [3]:
!pip install japanize_matplotlib
!pip install seaborn
!pip install networkx

  Using cached japanize-matplotlib-1.1.3.tar.gz (4.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120294 sha256=6326a0e028f4aeafd82de60943983c8aa0c9c8416496866de9da9b5018bc12a1
  Stored in directory: c:\users\irsl27\appdata\local\pip\cache\wheels\83\97\6b\e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize_matplotlib
     ------------------------------------- 293.3/293.3 kB 55.2 kB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 319.5 kB/s eta 0:00:00


In [6]:
input_str=input("意味類似度ネットワークを作りたい単語を入力してください===>")

In [7]:
from gensim.models import Word2Vec

# 学習済みモデルのロード
model_file_path = '../../KITERETU/gw2v160.model'
model = Word2Vec.load(model_file_path)

In [8]:
# モデル内に登録されている単語リストの長さをlen関数で見る（＝単語数）
print(len(model.wv.key_to_index))

# 「赤」に似ている単語TOP7を書き出す
out = model.wv.most_similar(positive=[input_str], topn=10)
print(out)

293753
[('青', 0.8831947445869446), ('黄色', 0.7592792510986328), ('白', 0.7574406862258911), ('青色', 0.7550033926963806), ('黒', 0.7461652159690857), ('ずきん', 0.7437013983726501), ('赤青', 0.7349072694778442), ('赤色', 0.7316375970840454), ('水色', 0.7268540859222412), ('赤白', 0.7261124849319458)]


In [9]:
print(out)

#AtCoderでよく使う辞書処理を使って県と数値を取り出す
pre=[]
pre.append(input_str)
for prefecture, value in out:
    print(prefecture, value)
    pre.append(prefecture)

[('青', 0.8831947445869446), ('黄色', 0.7592792510986328), ('白', 0.7574406862258911), ('青色', 0.7550033926963806), ('黒', 0.7461652159690857), ('ずきん', 0.7437013983726501), ('赤青', 0.7349072694778442), ('赤色', 0.7316375970840454), ('水色', 0.7268540859222412), ('赤白', 0.7261124849319458)]
青 0.8831947445869446
黄色 0.7592792510986328
白 0.7574406862258911
青色 0.7550033926963806
黒 0.7461652159690857
ずきん 0.7437013983726501
赤青 0.7349072694778442
赤色 0.7316375970840454
水色 0.7268540859222412
赤白 0.7261124849319458


実装方式に教授の考え方をそのまま応用<br>

それぞれのネットワーク的な接続を数値で表す。<br>
表し方としては、2次元配列を用いる<br>

In [10]:
import pandas as pd

df = pd.crosstab(pre, pre, rownames=['X'], colnames=['Y'])
df

AssertionError: arrays and names must have the same length

In [11]:
for i in pre:
    for j in pre:
        #0を除外する処理を入れていないので計算数がその分多い
        if model.wv.similarity(i,j) >= 1.0:
            df.loc[i,j]=0.0
        elif model.wv.similarity(i,j) >= 0.6:
            df.loc[i,j]=model.wv.similarity(i,j)
        else:
            df.loc[i,j]=0
        #print(df.loc['Bob', 'age'])
        #df.iloc[1,0]
df

NameError: name 'df' is not defined

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import japanize_matplotlib


plt.figure()
plt.title("入力文字列「{}」により想起された文字列の類似度ヒートマップ".format(input_str))
sns.heatmap(df,cmap="winter")
plt.savefig('eaborn_heatmap_dataframe.png')

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# ネットワークグラフの作成
G = nx.from_pandas_adjacency(df, create_using=nx.Graph)

# グラフの描画
plt.figure(figsize=(len(input_str)+8, len(input_str)+6))
pos = nx.spring_layout(G)  # ノードの位置を設定
weights = nx.get_edge_attributes(G, 'weight')

# 辺の太さを重みに応じて調整
max_weight = max(weights.values())
edge_widths = [(6 * (1 - weights[edge])) ** 2 for edge in G.edges()]

# ノードの太さを重みに応じて調整
node_sizes = [50 + 150 * G.degree(node) for node in G.nodes()]

# グラフを描画
japanize_matplotlib.japanize()
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=node_sizes)
nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color='black')
nx.draw_networkx_labels(G, pos, font_family='IPAexGothic', font_color='black', font_size=10)
plt.title('ネットワークグラフ')
plt.axis('off')
plt.show()
